In [12]:
from sympy import *
import ipywidgets as widgets

# Predimensionamiento (Datos Generales)

In [13]:
esf_adm=3.15 #kgf/cm2
dens_suelo=1.75 #tnf/m3
dens_concreto=2.4 #tnf/m3
ang_friccion=30
mu=tan(2/3*ang_friccion*pi/180)
sc1=1 #tnf/m2
sc2=0 #tnf/m2

In [14]:
c=widgets.FloatText(description='corona',value='1')
p=widgets.FloatText(description='pie',value='1')
t=widgets.FloatText(description='talon',value='1')
pz=widgets.FloatText(description='h_z',value='1')
lz=widgets.FloatText(description='l_z',value='1')
hm=widgets.FloatText(description='h_mc',value='1')
nfz=widgets.FloatText(description='nfz',value='1')
sc=widgets.FloatText(description='sc',value='1')
display(c,p,t,pz,lz,hm,nfz,sc)

FloatText(value=1.0, description='corona')

FloatText(value=1.0, description='pie')

FloatText(value=1.0, description='talon')

FloatText(value=1.0, description='h_z')

FloatText(value=1.0, description='l_z')

FloatText(value=1.0, description='h_mc')

FloatText(value=1.0, description='nfz')

FloatText(value=1.0, description='sc')

# 2.Empujes

## 2.1.Empuje Activo

In [31]:
ka=tan((45-ang_friccion/2)*pi/180)**2
pa=ka*dens_suelo*hm.value
Ea=pa*hm.value/2
print('El empuje activo es: ',round(Ea,2))

El empuje activo es:  7.29


## 2.2.Empuje Sobrecarga

In [32]:
Ea_sc1=ka*sc1*hm.value
Ea_sc2=ka*sc2*hm.value
Ea_sc=Ea_sc1+Ea_sc2
print('El empuje activo de la sobrecarga es: ',round(Ea_sc,2))

El empuje activo de la sobrecarga es:  7.50


# 2.3.Empuje Pasivo

In [33]:
kp=tan((45+ang_friccion/2)*pi/180)**2
pp=kp*dens_suelo*pz.value
Ep=pp*pz.value/2
print('El empuje pasivo es: ',round(Ep,2))

El empuje pasivo es:  1.68


# 3.Estabilidad

## 3.1.Volteo

In [34]:
#Momento de Equilibrio
#Concreto
z1=(pz.value*lz.value*1)*dens_concreto
br1=lz.value/2
z2=(c.value*hm.value*1)*dens_concreto
br2=c.value/2+p.value
z3=((lz.value-(p.value+t.value+c.value))*hm.value/2*1)*dens_concreto
br3=1/3*(lz.value-(p.value+t.value+c.value))+c.value+p.value
#Suelo
z4=(t.value*hm.value*1)*dens_suelo
br_z4=lz.value-t.value/2
z5=((lz.value-(p.value+t.value+c.value))*hm.value/2*1)*dens_suelo
br_z5=2/3*(lz.value-(p.value+t.value+c.value))+c.value+p.value
z6=sc.value*((lz.value-p.value-c.value)*1)
br_z6=(lz.value-c.value-p.value)/2
Me=z1*br1+z2*br2+z3*br3+z4*br_z4+z5*br_z5+z6*br_z6
print('Momento de Equilibrio=',round(Me,2),'tnf-m')
#Momento de Volteo
Mv=Ea*(hm.value/3+pz.value)+Ea_sc*(hm.value/2+pz.value)
print('Momento de Volteo=',round(Mv,2),'tnf-m')
#Factor de Seguridad
FS_v=Me/Mv
print('FS de Volteo=',round(FS_v,2),'tnf-m')

Momento de Equilibrio= 43.02 tnf-m
Momento de Volteo= 42.74 tnf-m
FS de Volteo= 1.01 tnf-m


## 3.2.Deslizamiento

In [35]:
#Deslizamiento Activo
Da=Ea+Ea_sc
#Deslizamiento Resistente
N=z1+z2+z3+z4+z5+z6
Dr=mu*N+Ep
if Da>Dr:
  print('Necesita Dentellon')
  print('Da=',round(Da,2),'>','Dr=',round(Dr,2))
else:
  print('No Necesita Dentellon')
  print('Da=',round(Da,2),'<','Dr=',round(Dr,2))
FS_d=Dr/Da
print('FS de Volteo=',round(FS_d,2),'tnf-m')
x=symbols('x')
Hs=solveset(Eq((kp*x**2*dens_suelo)/2,(1.5*Da-mu*N)),x)
hs1,hs2=Hs
Hd=hs1-pz.value
print('Altura Dentellon=',round(Hd,2),'m')

Necesita Dentellon
Da= 14.79 > Dr= 10.09
FS de Volteo= 0.68 tnf-m
Altura Dentellon= 1.49 m


## 3.3.Esfuerzos Admisibles

In [36]:
P=z1+z2+z3+z4+z5+z6
x2=(Me-Mv)/P
M=P*(lz.value/2-x2)
print('P=',round(P,2))
print('x2=',round(x2,2))
print('M=',round(M,2))
A=lz.value*1
y=lz.value/2
I=1*(lz.value)**3/12
sigma_p=P/A+M*y/I
sigma_n=P/A-M*y/I
print('sigma_p=',round(sigma_p,2))
print('sigma_n=',round(sigma_n,2))
sigma_m=2/3*(P/(lz.value/2-x2))
if sigma_m<esf_adm*10*1.3:
  print('sigma_m=',sigma_m.evalf(3),'>','esf_adm=',round(esf_adm*10*1.3,2))
  print('ok')
else:
  print('sigma_m=',sigma_m.evalf(3),'<','esf_adm=',round(esf_adm*10*1.3,2))
  print('no ok')

P= 23.11
x2= 0.01
M= 34.38
sigma_p= 30.63
sigma_n= -15.22
sigma_m= 10.4 > esf_adm= 40.95
ok


# 4.Esfuerzos internos

Revisar los calculos...